<a href="https://colab.research.google.com/github/SagnikPanda38/demo/blob/main/Copy_of_ReinforcementLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stable-baselines3[extra]
!pip uninstall -y gym
!pip install gymnasium
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

environment_name = "CartPole-v0"
env = gym.make(environment_name, render_mode="human") # Added render_mode
episodes = 5
for episode in range(1, episodes+1):
    state, info = env.reset() # reset returns two values in gymnasium
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, terminated, truncated, info = env.step(action) # gymnasium returns terminated and truncated
        done = terminated or truncated # Update done condition
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()


# 0-push cart to left, 1-push cart to the right
env.action_space.sample()
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

env = gym.make(environment_name) # Re-initialize for PPO, render_mode not needed for DummyVecEnv
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)
model.learn(total_timesteps=20000)

Found existing installation: gym 0.25.2
Uninstalling gym-0.25.2:
  Successfully uninstalled gym-0.25.2


/usr/local/lib/python3.12/dist-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


Episode:1 Score:15.0
Episode:2 Score:15.0
Episode:3 Score:48.0
Episode:4 Score:21.0
Episode:5 Score:27.0
Using cpu device
-----------------------------
| time/              |      |
|    fps             | 1187 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


-----------------------------------------
| time/                   |             |
|    fps                  | 940         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008461937 |
|    clip_fraction        | 0.109       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00347     |
|    learning_rate        | 0.0003      |
|    loss                 | 7.37        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0176     |
|    value_loss           | 50.4        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 879         |
|    iterations           | 3           |
|    time_elapsed         | 6           |
|    total_timesteps      | 6144  

In [ ]:
import os
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')
model.save(PPO_path)

/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path 'Training/Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [ ]:
del model

In [ ]:
model = PPO.load(PPO_path, env = env)

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/vec_env/base_vec_env.py:259: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(np.float64(200.0), np.float64(0.0))

In [ ]:
env.close()

In [ ]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done:
        print('info', info)
        break

info [{'TimeLimit.truncated': True, 'terminal_observation': array([1.0191419e+00, 2.9832056e-01, 5.7936594e-04, 4.3159196e-01],
      dtype=float32)}]


In [ ]:
env.close

<bound method DummyVecEnv.close of <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7c3a65f34ec0>>

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=save_path,
                             verbose=1)
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)
model.learn(total_timesteps=20000, callback=eval_callback)
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

Using cpu device
Logging to Training/Logs/PPO_1


/usr/local/lib/python3.12/dist-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


-----------------------------
| time/              |      |
|    fps             | 1215 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 948         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008329038 |
|    clip_fraction        | 0.0811      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.00116    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.49        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0124     |
|    value_loss           | 51.9        |
-----------------------------------------
----------------------------------

/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=193.80 +/- 12.40
Episode length: 193.80 +/- 12.40
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 194         |
|    mean_reward          | 194         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.006182948 |
|    clip_fraction        | 0.0448      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.617      |
|    explained_variance   | 0.19        |
|    learning_rate        | 0.0003      |
|    loss                 | 30          |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0117     |
|    value_loss           | 74.1        |
-----------------------------------------
New best mean reward!
Stopping training because the mean reward 193.80 is above the threshold 190


/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/vec_env/base_vec_env.py:259: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


In [ ]:


training_log_path = os.path.join(log_path, 'PPO_3')
!tensorboard --logdir={training_log_path}


/usr/local/lib/python3.12/dist-packages/tensorboard/_vendor/bleach/sanitizer.py:292: SyntaxWarning: invalid escape sequence '\s'
  "[`\000-\040\177-\240\s]+",
/usr/local/lib/python3.12/dist-packages/tensorboard/_vendor/bleach/sanitizer.py:339: SyntaxWarning: invalid escape sequence '\s'
  style = re.compile('url\s*\(\s*[^\s)]+?\s*\)\s*').sub(' ', style)
/usr/local/lib/python3.12/dist-packages/tensorboard/_vendor/bleach/sanitizer.py:354: SyntaxWarning: invalid escape sequence '\s'
  if not re.match("^\s*([-\w]+\s*:[^:;]*(;\s*|$))*$", style):
/usr/local/lib/python3.12/dist-packages/tensorboard/_vendor/bleach/sanitizer.py:358: SyntaxWarning: invalid escape sequence '\w'
  for prop, value in re.findall('([-\w]+)\s*:\s*([^:;]*)', style):
2025-12-24 10:54:49.438923: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766573689.461971    3691 cuda_dnn.

In [ ]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})
model.learn(total_timesteps=20000, callback=eval_callback)

Using cpu device


/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 1078 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 752         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015038948 |
|    clip_fraction        | 0.19        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | -0.0072     |
|    learning_rate        | 0.0003      |
|    loss                 | 2.6         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.022      |
|    value_loss           | 18.4        |
-----------------------------------------
----------------------------------

In [ ]:
from stable_baselines3 import DQN
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)
model.learn(total_timesteps=20000, callback=eval_callback)
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')
model.save(dqn_path)
model = DQN.load(dqn_path, env=env)
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

Streaming output truncated to the last 5000 lines.
|    learning_rate    | 0.0001   |
|    loss             | 0.000576 |
|    n_updates        | 1289     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 464      |
|    fps              | 974      |
|    time_elapsed     | 5        |
|    total_timesteps  | 5295     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000239 |
|    n_updates        | 1298     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 468      |
|    fps              | 975      |
|    time_elapsed     | 5        |
|    total_timesteps  | 5338     |
| train/              |          |
|    learning_rate    | 0.0001   |
|   